In [5]:
# TOKEN HIDDEN
SPORT_ID  = 18
LEAGUE_ID = 26537  # Brazil LDB U22

In [8]:
import requests
import json
import pandas as pd
import time
from datetime import datetime

In [9]:
# Extrair dados de todos os quartos
SPORT_ID = 18
API_URL   = "https://api.b365api.com/v3/events/ended"
TOKEN = API_TOKEN = "183604-pWN7flhoAsWGu8"


def fetch_ended_events(sport_id: int, league_id: int, token: str):
    all_events = []
    page = 1
    while True:
        params = {"sport_id": sport_id, "league_id": league_id, "token": token, "page": page}
        resp = requests.get(API_URL, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json().get("results", [])
        if not data:
            break
        all_events.extend(data)
        print(f"Página {page}: {len(data)} eventos carregados")
        page += 1
        time.sleep(0.2)
    return all_events

if __name__ == "__main__":
    events = fetch_ended_events(SPORT_ID, LEAGUE_ID, TOKEN)
    print(f"\nTotal de eventos retornados: {len(events)}")

    rows = []
    for evt in events:
        ev_id = evt.get("id")
        ts    = evt.get("time")
        date  = datetime.fromtimestamp(int(ts)).strftime("%Y-%m-%d %H:%M:%S") if ts else None
        home  = evt.get("home", {}).get("name")
        away  = evt.get("away", {}).get("name")

        # final a partir de ss
        ss = evt.get("ss", "")
        if ss and "-" in ss:
            home_final, away_final = ss.split("-", 1)
        else:
            home_final = away_final = None

        # faz uma cópia das chaves de período, ordena e retira o último (que também é o final)
        scores = evt.get("scores", {}) or {}
        period_keys = sorted(scores.keys(), key=lambda k: int(k))
        if not period_keys:
            continue
        period_keys.pop()  # ignora sempre o último

        row = {
            "event_id":  ev_id,
            "date":      date,
            "home_team": home,
            "away_team": away,
        }

        # mapeia Q1, Q2, HT, Q3, Q4
        if "1" in period_keys:
            row["home_q1"], row["away_q1"] = scores["1"]["home"], scores["1"]["away"]
        if "2" in period_keys:
            row["home_q2"], row["away_q2"] = scores["2"]["home"], scores["2"]["away"]
        if "3" in period_keys:
            row["home_ht"], row["away_ht"] = scores["3"]["home"], scores["3"]["away"]
        if "4" in period_keys:
            row["home_q3"], row["away_q3"] = scores["4"]["home"], scores["4"]["away"]
        if "5" in period_keys:
            row["home_q4"], row["away_q4"] = scores["5"]["home"], scores["5"]["away"]

        # prorrogações reais (se houver)
        ot_keys = [k for k in period_keys if int(k) > 5]
        ot_keys.sort(key=lambda k: int(k))
        for idx, k in enumerate(ot_keys, start=1):
            row[f"home_ot{idx}"], row[f"away_ot{idx}"] = scores[k]["home"], scores[k]["away"]

        # coloca o final na coluna de OT seguinte às prorrogações
        next_ot = len(ot_keys) + 1
        row[f"home_ot{next_ot}"]  = home_final
        row[f"away_ot{next_ot}"]  = away_final

        # mantém também as colunas finais dedicadas
        row["home_final"] = home_final
        row["away_final"] = away_final

        rows.append(row)

    df = pd.DataFrame(rows)

    # reordena para garantir que final fique sempre no fim
    cols_sem_final = [c for c in df.columns if c not in ("home_final","away_final")]
    df = df[cols_sem_final + ["home_final","away_final"]]

    df.to_csv("placares_ldb_todos_quartos.csv", index=False, encoding="utf-8")
    print("Dados salvos em placares_ldb_todos_quartos.csv")


Página 1: 50 eventos carregados
Página 2: 50 eventos carregados
Página 3: 50 eventos carregados
Página 4: 50 eventos carregados
Página 5: 50 eventos carregados
Página 6: 50 eventos carregados
Página 7: 50 eventos carregados
Página 8: 50 eventos carregados
Página 9: 50 eventos carregados
Página 10: 50 eventos carregados
Página 11: 50 eventos carregados
Página 12: 50 eventos carregados
Página 13: 50 eventos carregados
Página 14: 12 eventos carregados

Total de eventos retornados: 662
Dados salvos em placares_ldb_todos_quartos.csv


In [3]:
# Puxar Odds de eventos LDB
import pandas as pd
import requests
import time
import sys

# 1) Carrega o CSV original
input_csv  = "resultados_nbb_20241012_20250618.csv"
df         = pd.read_csv(input_csv)

# Prepara colunas novas, já preenchidas com NaN
new_cols   = ["odd_id", "handicap", "home_od", "away_od", "add_time", "time_str"]
for c in new_cols:
    df[c] = pd.NA

# 2) Configuração da API
API_TOKEN = "183604-pWN7flhoAsWGu8"
URL       = "https://api.b365api.com/v2/event/odds"

def fetch_latest_null_ss(event_id):
    params = {"token": API_TOKEN, "event_id": event_id}
    r = requests.get(URL, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    odds = data.get("results", {}).get("odds", {}).get("18_2", [])
    null_ss = [o for o in odds if o.get("ss") is None]
    if not null_ss:
        return None
    return max(null_ss, key=lambda o: int(o["add_time"]))

# 3) Itera e preenche o DataFrame
for idx, row in df.iterrows():
    eid = row["event_id"]
    try:
        item = fetch_latest_null_ss(eid)
        if item:
            # percorre as colunas e atribui ao DataFrame
            df.at[idx, "handicap"] = item["handicap"]
            df.at[idx, "home_od"]  = item["home_od"]
            df.at[idx, "away_od"]  = item["away_od"]
            print(f"event_id {eid}: OK (add_time={item['add_time']})")
        else:
            print(f"event_id {eid}: sem odds ss=null")
    except Exception as e:
        print(f"event_id {eid}: ERRO → {e}", file=sys.stderr)
    time.sleep(0.2)  # para não estourar rate-limit

# 4) Salva o CSV com as colunas originais + odds
output_csv = "resultados_com_odds.csv"
df.to_csv(output_csv, index=False)
print(f"\nSalvo CSV final em: {output_csv}")


FileNotFoundError: [Errno 2] No such file or directory: 'resultados_nbb_20241012_20250618.csv'

In [10]:
# Puxar Odds

# 1) Carrega o CSV original
input_csv  = "placares_ldb_todos_quartos.csv"
df         = pd.read_csv(input_csv)

# 2) Prepara colunas novas
# — para mercado 18_2 (manter nomes originais)
cols_2 = ["odd_id", "handicap", "home_od", "away_od", "add_time", "time_str"]
# — para mercado 18_3 (over/under)
cols_3 = ["odd_id_3", "handicap_3", "over_od_3", "under_od_3", "add_time_3", "time_str_3"]
# — para mercado 18_8 (q=0)
cols_8 = ["odd_id_8", "handicap_8", "home_od_8", "away_od_8", "add_time_8", "time_str_8"]

for c in cols_2 + cols_3 + cols_8:
    df[c] = pd.NA

# 3) Configuração da API
API_TOKEN = "183604-pWN7flhoAsWGu8"
URL       = "https://api.b365api.com/v2/event/odds"

def fetch_latest_odd(event_id, market, od_keys,
                     require_ss_null=True, q_filter=None):
    """
    Busca odds para um dado mercado, filtra por:
      - ss=None (se require_ss_null=True)
      - q == q_filter (se q_filter não for None)
      - valores de odd != '-' para cada chave em od_keys
    Retorna o item com maior add_time, ou None se não houver nenhum.
    """
    params = {"token": API_TOKEN, "event_id": event_id}
    r = requests.get(URL, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    odds = data.get("results", {}).get("odds", {}).get(market, [])
    
    # filtra ss null
    if require_ss_null:
        odds = [o for o in odds if o.get("ss") is None]
    # filtra q, se solicitado
    if q_filter is not None:
        odds = [o for o in odds if str(o.get("q")) == str(q_filter)]
    # filtra '-' nas odds
    for k in od_keys:
        odds = [o for o in odds if o.get(k) not in (None, "-", "")]
    if not odds:
        return None
    
    # retorna o mais recente
    return max(odds, key=lambda o: int(o["add_time"]))

# 4) Itera e preenche o DataFrame
for idx, row in df.iterrows():
    eid = row["event_id"]
    try:
        # — mercado 18_2 (home/away, ss null)
        item2 = fetch_latest_odd(eid, "18_2", ["home_od", "away_od"])
        if item2:
            df.at[idx, "odd_id"]   = item2["id"]
            df.at[idx, "handicap"] = item2["handicap"]
            df.at[idx, "home_od"]  = item2["home_od"]
            df.at[idx, "away_od"]  = item2["away_od"]
            df.at[idx, "add_time"] = item2["add_time"]
            df.at[idx, "time_str"] = item2["time_str"]
        else:
            print(f"event_id {eid}: sem odds válidas 18_2")

        # — mercado 18_3 (over/under, ss null)
        item3 = fetch_latest_odd(eid, "18_3", ["over_od", "under_od"])
        if item3:
            df.at[idx, "odd_id_3"]     = item3["id"]
            df.at[idx, "handicap_3"]   = item3["handicap"]
            df.at[idx, "over_od_3"]    = item3["over_od"]
            df.at[idx, "under_od_3"]   = item3["under_od"]
            df.at[idx, "add_time_3"]   = item3["add_time"]
            df.at[idx, "time_str_3"]   = item3["time_str"]
        else:
            print(f"event_id {eid}: sem odds válidas 18_3")

        # — mercado 18_8 (home/away, pega q=0, não exige ss null)
        item8 = fetch_latest_odd(eid, "18_8", ["home_od", "away_od"],
                                 require_ss_null=False, q_filter=0)
        if item8:
            df.at[idx, "odd_id_8"]     = item8["id"]
            df.at[idx, "handicap_8"]   = item8["handicap"]
            df.at[idx, "home_od_8"]    = item8["home_od"]
            df.at[idx, "away_od_8"]    = item8["away_od"]
            df.at[idx, "add_time_8"]   = item8["add_time"]
            df.at[idx, "time_str_8"]   = item8["time_str"]
        else:
            print(f"event_id {eid}: sem odds válidas 18_8 (q=0)")

    except Exception as e:
        print(f"event_id {eid}: ERRO → {e}", file=sys.stderr)

    time.sleep(0.2)  # evita rate-limit

# 5) Salva o CSV final
output_csv = "resultados_com_odds TESTE.csv"
df.to_csv(output_csv, index=False)
print(f"\nSalvo CSV final em: {output_csv}")


event_id 10384511: sem odds válidas 18_8 (q=0)

Salvo CSV final em: resultados_com_odds TESTE.csv


In [1]:
# Puxar Missing Odds
import pandas as pd
import requests
import time
import sys

# 1) Carrega o CSV original
input_csv  = 'missing_odds.csv'
df         = pd.read_csv(input_csv)

# 2) Prepara colunas novas
cols_2 = ["odd_id", "handicap", "home_od", "away_od", "add_time", "time_str"]
cols_3 = ["odd_id_3", "handicap_3", "over_od_3", "under_od_3", "add_time_3", "time_str_3"]
cols_8 = ["odd_id_8", "handicap_8", "home_od_8", "away_od_8", "add_time_8", "time_str_8"]

for c in cols_2 + cols_3 + cols_8:
    df[c] = pd.NA

# 3) Configuração da API
API_TOKEN = "183604-pWN7flhoAsWGu8"
URL       = "https://api.b365api.com/v2/event/odds"

def fetch_latest_odd(event_id, market, od_keys,
                     require_ss_null=True, q_filter=None):
    params = {"token": API_TOKEN, "event_id": event_id}
    r = requests.get(URL, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    odds = data.get("results", {}).get("odds", {}).get(market, [])
    
    if require_ss_null:
        odds = [o for o in odds if o.get("ss") is None]
    if q_filter is not None:
        odds = [o for o in odds if str(o.get("q")) == str(q_filter)]
    for k in od_keys:
        odds = [o for o in odds if o.get(k) not in (None, "-", "")]
    if not odds:
        return None
    
    return max(odds, key=lambda o: int(o["add_time"]))

# define arquivo de saída antes do loop, para poder salvar progresso
output_csv = "resultados_com_odds_faltantes.csv"

# 4) Itera e preenche o DataFrame
for idx, row in df.iterrows():
    eid = row["event_id"]
    try:
        # — mercado 18_2 (home/away, ss null)
        item2 = fetch_latest_odd(eid, "18_2", ["home_od", "away_od"])
        if item2:
            df.at[idx, "odd_id"]   = item2["id"]
            df.at[idx, "handicap"] = item2["handicap"]
            df.at[idx, "home_od"]  = item2["home_od"]
            df.at[idx, "away_od"]  = item2["away_od"]
            df.at[idx, "add_time"] = item2["add_time"]
            df.at[idx, "time_str"] = item2["time_str"]
        else:
            print(f"event_id {eid}: sem odds válidas 18_2")

        # — mercado 18_3 (over/under, ss null)
        item3 = fetch_latest_odd(eid, "18_3", ["over_od", "under_od"])
        if item3:
            df.at[idx, "odd_id_3"]     = item3["id"]
            df.at[idx, "handicap_3"]   = item3["handicap"]
            df.at[idx, "over_od_3"]    = item3["over_od"]
            df.at[idx, "under_od_3"]   = item3["under_od"]
            df.at[idx, "add_time_3"]   = item3["add_time"]
            df.at[idx, "time_str_3"]   = item3["time_str"]
        else:
            print(f"event_id {eid}: sem odds válidas 18_3")

        # — mercado 18_8 (home/away, pega q=0, não exige ss null)
        item8 = fetch_latest_odd(eid, "18_8", ["home_od", "away_od"],
                                 require_ss_null=False, q_filter=0)
        if item8:
            df.at[idx, "odd_id_8"]     = item8["id"]
            df.at[idx, "handicap_8"]   = item8["handicap"]
            df.at[idx, "home_od_8"]    = item8["home_od"]
            df.at[idx, "away_od_8"]    = item8["away_od"]
            df.at[idx, "add_time_8"]   = item8["add_time"]
            df.at[idx, "time_str_8"]   = item8["time_str"]
        else:
            print(f"event_id {eid}: sem odds válidas 18_8 (q=0)")

    except Exception as e:
        print(f"event_id {eid}: ERRO → {e}", file=sys.stderr)

    # **salva progresso após cada evento**, para não perder tudo em caso de erro
    df.to_csv(output_csv, index=False)

    time.sleep(0)  # evita rate-limit

# 5) Salva o CSV final
print(f"\nSalvo CSV final em: {output_csv}")


FileNotFoundError: [Errno 2] No such file or directory: 'missing_odds.csv'

In [ ]:
df = pd.read_csv("resultados_com_odds.csv")

In [ ]:
df2 = pd.read_csv('resultados_com_odds_faltantes.csv')

In [ ]:
df = df.dropna(subset=['handicap'])     

In [ ]:
df2 = df2.dropna(subset=['handicap'])

In [ ]:
df_final = pd.concat([df, df2], ignore_index=True)

In [6]:
import pandas as pd
import requests
import json
import time
import sys

# 1) Carrega o CSV original
input_csv  = "placares_ldb_todos_quartos copy.csv"
df         = pd.read_csv(input_csv)

# 2) Configuração da API
API_TOKEN = "183604-pWN7flhoAsWGu8"
URL       = "https://api.b365api.com/v2/event/odds"

# 3) Função para buscar todas as odds de um evento
def fetch_all_odds(event_id):
    """
    Retorna um dicionário com todos os mercados de odds
    para o event_id informado, ou None em caso de erro.
    """
    params = {"token": API_TOKEN, "event_id": event_id}
    try:
        r = requests.get(URL, params=params, timeout=10)
        r.raise_for_status()
    except Exception as e:
        print(f"event_id {event_id}: ERRO na requisição → {e}", file=sys.stderr)
        return None

    data = r.json()
    # Extrai o dict de mercados de odds (chave "odds")
    return data.get("results", {}).get("odds", {})

# 4) Itera sobre o DataFrame, busca e imprime o JSON de odds
for idx, row in df.iterrows():
    eid = row["event_id"]
    odds_dict = fetch_all_odds(eid)
    if odds_dict is None:
        continue

    print(f"\n--- event_id {eid} ---")
    # Imprime o JSON formatado, com todos os mercados e suas odds
    print(json.dumps(odds_dict, indent=2, ensure_ascii=False))

    # Pequena pausa para não estourar rate limit
    time.sleep(0.2)



--- event_id 10209829 ---
{
  "18_2": [
    {
      "id": "149800502",
      "home_od": "-",
      "handicap": "-13.5",
      "away_od": "-",
      "ss": "66:53",
      "time_str": "4 - 00:00",
      "add_time": "1751582930"
    },
    {
      "id": "149800493",
      "home_od": "-",
      "handicap": "-12.5",
      "away_od": "-",
      "ss": "66:53",
      "time_str": "4 - 00:24",
      "add_time": "1751582925"
    },
    {
      "id": "149800490",
      "home_od": "-",
      "handicap": "-13.5",
      "away_od": "-",
      "ss": "66:53",
      "time_str": "4 - 00:08",
      "add_time": "1751582922"
    },
    {
      "id": "149800487",
      "home_od": "-",
      "handicap": "-12.5",
      "away_od": "-",
      "ss": "66:53",
      "time_str": "4 - 00:24",
      "add_time": "1751582920"
    },
    {
      "id": "149800476",
      "home_od": "-",
      "handicap": "-13.5",
      "away_od": "-",
      "ss": "66:53",
      "time_str": "4 - 00:19",
      "add_time": "1751582907"
    },